In [99]:
import pandas as pd
import numpy as np
#!pip install matplotlib
#!pip install seaborn
import seaborn as sns

import numpy as np
import gym
from stable_baselines3 import PPO
import math
import itertools
import datetime
import ray
import ray.rllib as rllib
from ray import tune
from ray.tune.registry import register_env


In [18]:
#!pip install gym==0.17.3

In [19]:
df = pd.read_csv('../data/dist_vologda_matrix.csv', index_col='Unnamed: 0')

In [20]:
df.values.max()
#585822.9
#794596.1

794596.1

In [94]:
test_df = df[:5][df.columns[:5]]
test_df

,АЗС №35400 Кичменгский Городок,АЗС №35401 Великий Устюг-1,АЗС №35402 Великий Устюг-2,АЗС №35403 Дунай,АЗС №35404 Нюксеница
АЗС №35400 Кичменгский Городок,0.0,99227.2,101851.3,139213.5,154252.2
АЗС №35401 Великий Устюг-1,99068.4,0.0,4946.5,134520.1,149558.9
АЗС №35402 Великий Устюг-2,101814.8,5068.8,0.0,137266.5,152305.3
АЗС №35403 Дунай,139213.5,134679.1,137303.1,0.0,15096.1
АЗС №35404 Нюксеница,154252.2,149717.8,152341.9,15096.1,0.0


In [95]:
def get_min_dist(df):
    cols_done = []
    start_col = df.columns[-1]
    cols_done.append(start_col)
    dist = 0
    df2 = df.copy()
    for i in range(len(df.columns)-2):
        try:
            z = (df.loc[df.columns[-1]]).sort_values(ascending=False)[[-2]]
            start_col = z.index[0]
            cols_done.append(start_col)
            dist += z[0]
            feat = [col for col in df.columns if col not in cols_done]
            df = df[feat]
            print(dist)
        except:
            print('error')
            #print(z)
    #print([col for col in df2.columns if col not in cols_done])
    dist = dist + df2.loc[cols_done[-1], [col for col in df2.columns if col not in cols_done][0]]
    cols_done.append([col for col in df2.columns if col not in cols_done][0])
    print(dist)
    print(cols_done)
    #print(f'final dist: {}')
    #print(final_dist)
    #print(dist/(60*1000))
    
get_min_dist(test_df)

15096.1
20164.9
121979.70000000001
223831.0
['АЗС №35404 Нюксеница', 'АЗС №35403 Дунай', 'АЗС №35401 Великий Устюг-1', 'АЗС №35400 Кичменгский Городок', 'АЗС №35402 Великий Устюг-2']


In [ ]:
1444453

In [39]:
15096+134520.1+5068+101851.3+152341.9

408877.30000000005

In [33]:
class MyEnv(gym.Env):
    def __init__(self, env_config):
        super().__init__()
        self.n = N-1
        self.action_space = gym.spaces.Discrete(self.n)
        self.observation_space = gym.spaces.Dict({
            'visited': gym.spaces.MultiBinary(self.n), 
            'last': gym.spaces.Discrete(N)})   
        
    def reset(self):
        self.state = {'visited': np.zeros(self.n), 'last': 0}
        visited = np.zeros(self.n)
        return self.state

    def step(self, action):
        if self.state['visited'][action] == 1:
            self.reward = -170000
        else:
            self.state['visited'][action] = 1
            self.reward = - M[self.state['last'], action + 1]
            self.state['last'] = action + 1
        if np.all(self.state['visited'] == 1): 
            self.reward += - M[action + 1, 0]    
            self.done = True
        else:
            self.done = False
            
        return self.state, self.reward, self.done, {}


In [97]:
N = test_df.shape[0]
#rand = np.random.RandomState(1)
M = test_df.values

In [35]:
test_df.values

array([[     0. ,  99227.2, 101851.3, 139213.5, 154252.2],
       [ 99068.4,      0. ,   4946.5, 134520.1, 149558.9],
       [101814.8,   5068.8,      0. , 137266.5, 152305.3],
       [139213.5, 134679.1, 137303.1,      0. ,  15096.1],
       [154252.2, 149717.8, 152341.9,  15096.1,      0. ]])

In [36]:
config = rllib.agents.ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
config["framework"] = "torch"
config["env_config"] = {}


In [37]:
env = MyEnv(config)

In [38]:
print(datetime.datetime.now())
model = PPO("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=100000)
print(datetime.datetime.now())

2022-03-27 19:38:57.760605
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 8.63      |
|    ep_rew_mean     | -1.33e+06 |
| time/              |           |
|    fps             | 2446      |
|    iterations      | 1         |
|    time_elapsed    | 0         |
|    total_timesteps | 2048      |
----------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 7.99          |
|    ep_rew_mean          | -1.22e+06     |
| time/                   |               |
|    fps                  | 1681          |
|    iterations           | 2             |
|    time_elapsed         | 2             |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 1.1350494e-09 |
|    clip_fraction        | 0             |
|

In [31]:
obs = env.reset()
g = 0
actions = [0]
for i in range(10000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    g += M[actions[-1], action + 1]
    actions.append(action + 1)
    if done:
        obs = env.reset()
        g += M[actions[-1], 0]
        print(f"{actions}, g = {g}")
        break
env.close()


[0, 3, 4, 1, 2], g = 410788.7


In [32]:
g

410788.7

In [30]:
A = [1,2,3,4,5,6,7,8,9,9]
B = ['c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c']

In [93]:
def gr(A,B) -> dict:
    return dict(zip(B, A))
    
gr(A,B)

{'c': 9, 'a': 8, 'b': 9}

In [92]:
[ dict(zip(B, A))[i[0]] + i[1] for i in list(zip(B, A))]

[10, 10, 12, 13, 13, 15, 16, 16, 18, 18]

In [98]:
x_min = []
g_min = 100000000
print(datetime.datetime.now())
for i, x in enumerate(itertools.permutations(range(1,N), N-1)):
    x = (0,) + x
    g = np.sum(M[x[:-1], x[1:]])+M[x[-1], x[0]]
    if g < g_min:
        g_min = g
        x_min = x
    if i % 10000000 == 0:
        print(i, x_min, g_min)
print(datetime.datetime.now())
print('Optimal solution:')
print(x_min, g_min)



2022-03-31 18:57:26.528822
0 (0, 1, 2, 3, 4) 410788.5
2022-03-31 18:57:26.529962
Optimal solution:
(0, 1, 2, 3, 4) 410788.5


In [14]:
ray.shutdown()
ray.init()

2022-03-27 19:21:01,272	INFO services.py:1412 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.81',
 'raylet_ip_address': '192.168.1.81',
 'redis_address': None,
 'object_store_address': '/tmp/ray/session_2022-03-27_19-20-59_505779_483540/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-03-27_19-20-59_505779_483540/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-03-27_19-20-59_505779_483540',
 'metrics_export_port': 42802,
 'gcs_address': '192.168.1.81:60801',
 'address': '192.168.1.81:60801',
 'node_id': '882092a165829b0f043be997b0d502803f13b6c4106189f241f44a4d'}

In [15]:
agent = rllib.agents.ppo.PPOTrainer(config=config, env=MyEnv)

2022-03-27 19:21:02,679	INFO ppo.py:249 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-03-27 19:21:02,680	INFO trainer.py:779 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2022-03-27 19:21:05,117	WARNING util.py:55 -- Install gputil for GPU system monitoring.


In [16]:
print(datetime.datetime.now())
for i in range(31):
    # Perform one iteration of training the policy with PPO
    result = agent.train()
    if i % 10 == 0:
        #print(pretty_print(result))
        print('i: ', i)
        print('mean episode length:', result['episode_len_mean'])
        print('max episode reward:', result['episode_reward_max'])
        print('mean episode reward:', result['episode_reward_mean'])
        print('min episode reward:', result['episode_reward_min'])
        print('total episodes:', result['episodes_total'])
        print()
        
        checkpoint = agent.save()
print(datetime.datetime.now())

2022-03-27 19:21:05.122445


2022-03-27 19:21:11,891	WARNING ppo.py:162 -- The magnitude of your environment rewards are more than 120845.0x the scale of `vf_clip_param`. This means that it will take more than 120845.0 iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
2022-03-27 19:21:11,901	WARNING deprecation.py:45 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!


i:  0
mean episode length: 8.14256619144603
max episode reward: -410788.5
mean episode reward: -1208447.4293279022
min episode reward: -4677608.0
total episodes: 491



2022-03-27 19:21:18,619	WARNING ppo.py:162 -- The magnitude of your environment rewards are more than 97337.0x the scale of `vf_clip_param`. This means that it will take more than 97337.0 iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
2022-03-27 19:21:26,033	WARNING ppo.py:162 -- The magnitude of your environment rewards are more than 80100.0x the scale of `vf_clip_param`. This means that it will take more than 80100.0 iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
2022-03-27 19:21:33,340	WARNING ppo.py:162 -- The magnitude of your environment rewards are more than 64731.0x the scale of `vf_clip_param`. This means that it will take more than 64731.0 iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
2022-03-27 19:21:40,742	WARNING ppo.py:162 -- The magnitude of your environment rewards are more than 5528

i:  10
mean episode length: 4.02920443101712
max episode reward: -410788.5
mean episode reward: -420161.5818731117
min episode reward: -730788.6
total episodes: 9315



2022-03-27 19:22:30,768	WARNING ppo.py:162 -- The magnitude of your environment rewards are more than 41570.0x the scale of `vf_clip_param`. This means that it will take more than 41570.0 iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
2022-03-27 19:22:38,010	WARNING ppo.py:162 -- The magnitude of your environment rewards are more than 41407.0x the scale of `vf_clip_param`. This means that it will take more than 41407.0 iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
2022-03-27 19:22:45,317	WARNING ppo.py:162 -- The magnitude of your environment rewards are more than 41198.0x the scale of `vf_clip_param`. This means that it will take more than 41198.0 iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
2022-03-27 19:22:52,601	WARNING ppo.py:162 -- The magnitude of your environment rewards are more than 4142

i:  20
mean episode length: 4.002002002002002
max episode reward: -410788.5
mean episode reward: -411662.72622622625
min episode reward: -570788.7
total episodes: 19302



2022-03-27 19:23:42,576	WARNING ppo.py:162 -- The magnitude of your environment rewards are more than 41138.0x the scale of `vf_clip_param`. This means that it will take more than 41138.0 iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
2022-03-27 19:23:50,321	WARNING ppo.py:162 -- The magnitude of your environment rewards are more than 41111.0x the scale of `vf_clip_param`. This means that it will take more than 41111.0 iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
2022-03-27 19:23:57,914	WARNING ppo.py:162 -- The magnitude of your environment rewards are more than 41107.0x the scale of `vf_clip_param`. This means that it will take more than 41107.0 iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
2022-03-27 19:24:05,489	WARNING ppo.py:162 -- The magnitude of your environment rewards are more than 4114

i:  30
mean episode length: 4.0
max episode reward: -410788.5
mean episode reward: -410788.6436
min episode reward: -410788.7
total episodes: 29298

2022-03-27 19:24:52.170627


In [17]:
obs = env.reset()
g = 0
actions = [0]
for i in range(1000):
    action = agent.compute_single_action(obs, explore = False)
    obs, reward, done, info = env.step(action)
    g += M[actions[-1], action + 1]
    actions.append(action + 1)
    if done:
        obs = env.reset()
        g += M[actions[-1], 0]
        print(f"{actions}, g = {g}")
        break
env.close()

[0, 3, 4, 2, 1], g = 410788.69999999995


In [18]:
g

410788.69999999995

In [20]:
def env_creator(env_config):
    return MyEnv(env_config)  # return an env instance

register_env("my_env", env_creator)

In [21]:
experiment = tune.run(
    rllib.agents.ppo.PPOTrainer,
    config={
        "env": "my_env",
        "num_gpus": 0,
        "num_workers": 1,
        "framework": "torch"
    },
    metric="episode_reward_mean",
    mode="max",
    stop={"training_iteration": 50},
    checkpoint_at_end=True
)


Trial name,status,loc
PPOTrainer_my_env_91bec_00000,PENDING,


(PPOTrainer pid=483807) 2022-03-27 19:25:54,782	INFO ppo.py:249 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(PPOTrainer pid=483807) 2022-03-27 19:25:54,782	INFO trainer.py:779 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


Trial name,status,loc
PPOTrainer_my_env_91bec_00000,RUNNING,192.168.1.81:483807


(PPOTrainer pid=483807) 2022-03-27 19:25:57,470	WARNING util.py:55 -- Install gputil for GPU system monitoring.


Trial name,status,loc
PPOTrainer_my_env_91bec_00000,RUNNING,192.168.1.81:483807


...

(PPOTrainer pid=483807) 2022-03-27 19:32:29,168	WARNING ppo.py:162 -- The magnitude of your environment rewards are more than 41079.0x the scale of `vf_clip_param`. This means that it will take more than 41079.0 iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_my_env_91bec_00000,RUNNING,192.168.1.81:483807,49,390.782,196000,-410789,-410788,-410789,4


Result for PPOTrainer_my_env_91bec_00000:
  agent_timesteps_total: 200000
  custom_metrics: {}
  date: 2022-03-27_19-32-37
  done: true
  episode_len_mean: 4.003
  episode_media: {}
  episode_reward_max: -410788.5
  episode_reward_mean: -411268.5811
  episode_reward_min: -570788.6
  episodes_this_iter: 1000
  episodes_total: 48276
  experiment_id: 7faa3d15b44947ab8a2babd95c642244
  hostname: DK-GF63
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          allreduce_latency: 0.0
          cur_kl_coeff: 0.1351524353027344
          cur_lr: 5.0000000000000016e-05
          entropy: 0.41562690715635975
          entropy_coeff: 0.0
          kl: 0.010310875215937645
          policy_loss: -0.003798721521912563
          total_loss: 86797168666.42581
          vf_explained_var: -3.8646882580172634e-08
          vf_loss: 86797168666.42581
        model: {}
    num_agent_steps_sampled: 200000
    num_agent_steps_trained: 200000
    num_steps_sample

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_my_env_91bec_00000,TERMINATED,192.168.1.81:483807,50,398.695,200000,-411269,-410788,-570789,4.003


(PPOTrainer pid=483807) 2022-03-27 19:32:37,099	WARNING ppo.py:162 -- The magnitude of your environment rewards are more than 41127.0x the scale of `vf_clip_param`. This means that it will take more than 41127.0 iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
(PPOTrainer pid=483807) 2022-03-27 19:32:37,118	WARNING deprecation.py:45 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!
2022-03-27 19:32:37,820	INFO tune.py:639 -- Total run time: 405.74 seconds (405.04 seconds for the tuning loop).


In [22]:
agent.restore(experiment.best_checkpoint)

2022-03-27 19:32:37,876	INFO trainable.py:495 -- Restored on 192.168.1.81 from checkpoint: /home/kartushov/ray_results/PPOTrainer_2022-03-27_19-25-52/PPOTrainer_my_env_91bec_00000_0_2022-03-27_19-25-52/checkpoint_000050/checkpoint-50
2022-03-27 19:32:37,877	INFO trainable.py:503 -- Current state after restoring: {'_iteration': 50, '_timesteps_total': 200000, '_time_total': 398.69513988494873, '_episodes_total': 48276}


In [23]:
obs = env.reset()
g = 0
actions = [0]
for i in range(1000):
    action = agent.compute_single_action(obs, explore = False)
    obs, reward, done, info = env.step(action)
    g += M[actions[-1], action + 1]
    actions.append(action + 1)
    if done:
        obs = env.reset()
        g += M[actions[-1], 0]
        print(f"{actions}, g = {g}")
        break
env.close()


[0, 4, 3, 1, 2], g = 410788.7


In [24]:
g

410788.7

In [38]:
#actions

In [49]:
g

99227.2